In [8]:
from math import sqrt
import numpy as np
import pandas as pd 
import matplotlib

class Node:
	def __init__(self, df, feature, threshold, side):
		self.left=None
		self.right=None
		self.ds=df
		self.feature=feature
		self.threshold=threshold
		self.side=side
		self.count=df.iloc[:,22].count()
		self.label = -1

	def PrintTree(self, layer):
		pure = df.iloc[:,22]
		pure.drop_duplicates(keep="first",inplace=True)
		if (pure.count() > 1):
			if (self.side == 0):
				print("feature " + str(self.feature)+" <= " + str(self.threshold) + " at layer " + str(layer)+ " with data point: " + str(self.count))
			else:
				print("feature " + str(self.feature)+" > " + str(self.threshold) + " at layer " + str(layer)+ " with data point: " + str(self.count))
		else:
			print ("leaf label: " + str(self.ds.iloc[:,22][0]) + " at layer " + str(layer) + " with data point: " + str(self.count))
		layer+=1
		if self.left:
			self.left.PrintTree(self,layer)
		if self.right:
			self.right.PrintTree(self,layer)


def prob(df, feature, threshold, side):
	col = df.iloc[:,feature]
	if (side==0):
		return ((col)[col <= threshold].count()/col.count())
	else:
		return ((col)[col > threshold].count()/col.count())

def prob_cond(df, label, feature, threshold, side):
	col = df.iloc[:,[feature, 22]]
	#col = col.reset_index(drop = True)
	z = col.iloc[:,0]
	x = col.iloc[:,1]
	if (side == 0):
		return ((col)[(z <= threshold) & (x == label)].count()).iloc[1]/z[z <= threshold].count()
	else:		
		return ((col)[(z > threshold) & (x == label)].count()).iloc[1]/z[z > threshold].count()

def entropy(df):
	col = df.iloc[:,22]
	x0 = (col)[col == 0].count()/col.count()
	x1 = (col)[col == 1].count()/col.count()
	return -x0*np.log(x0)-x1*np.log(x1)

def entropy_cond(df, feature, threshold):
	col = df.iloc[:,[feature, 22]]
	z = col.iloc[:,0]
	x = col.iloc[:,1]
	pz0 = prob(df, feature, threshold, 0)
	pz1 = prob(df, feature, threshold, 1)
	px0z0 = prob_cond(df, 0, feature, threshold, 0)
	px1z0 = prob_cond(df, 1, feature, threshold, 0)
	px0z1 = prob_cond(df, 0, feature, threshold, 1)
	px1z1 = prob_cond(df, 1, feature, threshold, 1)
	if (px0z0==0): px0z0+=1
	if (px1z0==0): px1z0+=1
	if (px0z1==0): px0z1+=1
	if (px1z1==0): px1z1+=1
	ent = pz0*(-px0z0*np.log(px0z0)-px1z0*np.log(px1z0))+pz1*(-px0z1*np.log(px0z1)-px1z1*np.log(px1z1))
	return ent

def maxIG(df):
	minent = np.log(2)
	min_cond = [0,0]
	for i in range(22):
		temp = df.iloc[:,i]
		temp.drop_duplicates(keep="first",inplace=True)
		temp = temp.sort_values()
		temp = temp.reset_index(drop = True)
		for j in range(temp.count()-1):
			threshold = (temp[j]+temp[j+1])/2
			ent = entropy_cond(df, i, threshold)
			if (ent <= minent):
				minent = ent
				min_cond[0] = i
				min_cond[1] = threshold
	return min_cond

def ID3(df, current, layer):
	label = df.iloc[:,22]
	label.drop_duplicates(keep="first",inplace=True)
	label = label.reset_index(drop = True)
	if (label.count() == 1):
		current.label = label[0]
		print ("leaf label: " + str(label[0]) + " at layer " + str(layer) + " with data point: " + str(current.count))
		return 0
	if (label.count() > 1):
		cond = maxIG(current.ds)
		z = df.iloc[:,cond[0]]
		leftds = df[(z <= cond[1])]
		rightds = df[z > cond[1]]
		print("split conditon: feature "+ str(cond[0]) + " <=> " +str(cond[1]) + " at layer " + str(layer))
		current.left = Node(leftds, cond[0], cond[1], 0)
		current.right = Node(rightds, cond[0], cond[1], 1)
		ID3(leftds, current.left, layer+1)
		ID3(rightds, current.right, layer+1)
	return current

Location = r'C:\Users\SimonWCM\Downloads\HKU\Exchange\UC\Winter 2020\CSE151A\Programming HW2\pa2train.txt'
Location2 = r'C:\Users\SimonWCM\Downloads\HKU\Exchange\UC\Winter 2020\CSE151A\Programming HW2\pa2test.txt'
df = pd.read_csv(Location, sep = " ", header = None)
df2 = pd.read_csv(Location2, sep = " ", header = None)
#df.columns = ["LIMIT_BAL", "SEX", "COLLEGE_ABOVE", "AGE", "PAYMENT_DELAY_SEPTEMBER", "PAYMENT_DELAY_AUGUST", "PAYMENT_DELAY_JULY", "PAYMENT_DELAY_JUNE", "PAYMENT_DELAY_MAY", "PAYMENT_DELAY_APRIL", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5" ,"PAY_AMT6"]
#df2.columns = ["LIMIT_BAL", "SEX", "COLLEGE_ABOVE", "AGE", "PAYMENT_DELAY_SEPTEMBER", "PAYMENT_DELAY_AUGUST", "PAYMENT_DELAY_JULY", "PAYMENT_DELAY_JUNE", "PAYMENT_DELAY_MAY", "PAYMENT_DELAY_APRIL", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5" ,"PAY_AMT6"]
df = df.drop(columns=23)
df2 = df2.drop(columns=23)
print(df)
print(df2)
root = Node(df, 0, 0, 1)
root = ID3(df, root, 0)
print("****************end tree**************")

            0    1    2    3    4    5    6    7    8    9   ...        13  \
0      30000.0  2.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   30324.0   
1     160000.0  1.0  1.0  2.0  4.0  3.0  2.0  2.0  3.0  2.0  ...  103928.0   
2     190000.0  1.0  1.0  1.0  2.0  2.0  2.0  2.0  2.0  2.0  ...   38827.0   
3     140000.0  2.0  1.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    2880.0   
4     200000.0  2.0  1.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    2040.0   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...   
1995   60000.0  1.0  1.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   22816.0   
1996   30000.0  2.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   58287.0   
1997  200000.0  2.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  ...       0.0   
1998   50000.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...     390.0   
1999   20000.0  1.0  1.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   18357.0   

            14       15      16      17      18       19       

split conditon: feature 21 <=> 1280.5 at layer 8
leaf label: 0.0 at layer 9 with data point: 8
split conditon: feature 15 <=> 37210.0 at layer 9
split conditon: feature 20 <=> 1250.0 at layer 10
leaf label: 0.0 at layer 11 with data point: 1
leaf label: 1.0 at layer 11 with data point: 3
leaf label: 0.0 at layer 10 with data point: 4
split conditon: feature 0 <=> 115000.0 at layer 4
split conditon: feature 10 <=> 668.5 at layer 5
split conditon: feature 5 <=> 1.0 at layer 6
leaf label: 1.0 at layer 7 with data point: 7
split conditon: feature 19 <=> 224.5 at layer 7
leaf label: 0.0 at layer 8 with data point: 3
leaf label: 1.0 at layer 8 with data point: 1
split conditon: feature 16 <=> 1340.0 at layer 6
leaf label: 0.0 at layer 7 with data point: 10
split conditon: feature 16 <=> 1919.0 at layer 7
split conditon: feature 21 <=> 1050.0 at layer 8
split conditon: feature 21 <=> 279.5 at layer 9
leaf label: 1.0 at layer 10 with data point: 3
leaf label: 0.0 at layer 10 with data point: 3

split conditon: feature 14 <=> 100335.0 at layer 14
leaf label: 1.0 at layer 15 with data point: 5
leaf label: 0.0 at layer 15 with data point: 1
leaf label: 0.0 at layer 13 with data point: 10
leaf label: 0.0 at layer 11 with data point: 112
split conditon: feature 21 <=> 3544.5 at layer 8
split conditon: feature 20 <=> 44960.0 at layer 9
leaf label: 1.0 at layer 10 with data point: 4
leaf label: 0.0 at layer 10 with data point: 1
leaf label: 0.0 at layer 9 with data point: 6
split conditon: feature 18 <=> 4502.5 at layer 7
split conditon: feature 19 <=> 3557.0 at layer 8
split conditon: feature 21 <=> 1550.0 at layer 9
leaf label: 1.0 at layer 10 with data point: 1
leaf label: 0.0 at layer 10 with data point: 3
leaf label: 1.0 at layer 9 with data point: 6
split conditon: feature 18 <=> 9000.0 at layer 8
split conditon: feature 15 <=> 194572.5 at layer 9
leaf label: 0.0 at layer 10 with data point: 28
leaf label: 1.0 at layer 10 with data point: 1
split conditon: feature 13 <=> 19092

In [20]:
def searchNode(root, row):
    if (root.label<0):
        if (row[root.feature] <= root.threshold):
            return searchNode(root.left, row)
        else:
            return searchNode(root.right, row)
    else:
        return root.label
        

        
def testerror(df, root):
    predict = df.apply(lambda x: searchNode(root, x), axis=1)
    predict -= df.iloc[:,22]
    wrong = predict[predict!=0].count()
    return wrong/df.iloc[:,22].count()

training = testerror(df, root)
print("training error: " + str(training))
test = testerror(df2, root)
print("test error: " + str(test))
root.PrintTree(0)


training error: 0.285
test error: 0.272


TypeError: can only concatenate str (not "numpy.int32") to str

In [ ]:
def pruning(df, root):
    copy = root
    z = copy.ds.iloc[:,copy.feature]
    leftds = df[(z <= threshold)]
    leftds = leftds.iloc[:,22]
    if (leftds[leftds==0].count()>=leftds[leftds==1]):
        copy.left.label=0
    else:
        copy.left.label=1
    rightds = df[z > threshold]
    rightds = rightds.iloc[:,22]
    if (rightds[rightds==0].count()>=rightds[rightds==1]):
        copy.right.label=0
    else:
        copy.right.label=1
    if (testerror(df, root)>=testerror(df, copy)):
        root=copy
    return root

Location3 = r'C:\Users\SimonWCM\Downloads\HKU\Exchange\UC\Winter 2020\CSE151A\Programming HW2\pa2validation.txt'
df3 = pd.read_csv(Location3, sep = " ", header = None)
df3 = df3.drop(columns=23)
pruning(df3, root)
    
    